# FPSci Latency Study - Data Processing

This notebook processes raw experimental data from the latency study.

## Overview

- 27 participants
- 5 games (Fitts Law, Feeding Frenzy, Rocket League, Dave the Diver,
  Half-Life 2)
- 4 latency conditions (0, 75, 150, 225ms)
- Within-subjects design with Latin square counterbalancing


In [1]:
import json
import re
import pathlib
import warnings
from pathlib import Path
from itertools import product

import numpy as np
import pandas as pd
from IPython.display import display

warnings.filterwarnings("ignore")

# Set display options
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

## 1. Data Ingestion

Scan the experiment directory and catalog all available data.


In [2]:
# Define paths
EXPERIMENT_DIR = Path("../experiment")
OUTPUT_DIR = Path("../analysis/processed_data")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Game names mapping
GAME_NAMES = {
    "fitts_law": "Fitts Law",
    "feeding_frenzy": "Feeding Frenzy",
    "rocket_league": "Rocket League",
    "dave_the_diver": "Dave the Diver",
    "half_life_2": "Half-Life 2",
}

# Latency conditions
LATENCIES = [0, 75, 150, 225]

In [3]:
def extract_latency(round_name: str) -> int | None:
    """Extract latency from round directory name."""
    match = re.search(r"_(\d+)ms$", round_name)
    if match:
        return int(match.group(1))
    return None

def get_round_name_without_latency(round_name: str) -> str:
    return re.sub(r'_\d+ms$', '', round_name)


def is_test_round(round_name: str) -> bool:
    """Check if this is a test round."""
    return "_test" in round_name


def scan_experiment_data() -> pd.DataFrame:
    """Scan experiment directory and catalog all data."""
    data_inventory = []

    for participant_dir in sorted(EXPERIMENT_DIR.iterdir()):
        participant_id = int(participant_dir.name)
        # Find session directories (timestamped)
        session_dirs = [d for d in participant_dir.iterdir() if d.is_dir()]
        assert len(session_dirs) == 1, "every participant must have a session"

        for session_dir in session_dirs:
            session_timestamp = session_dir.name

            # Find game directories
            game_dirs = [d for d in session_dir.iterdir() if d.is_dir()]

            for game_dir in game_dirs:
                game_name = game_dir.name

                # Skip buggy directories
                if "BUG" in game_name:
                    continue

                # Find round directories
                round_dirs = [d for d in game_dir.iterdir() if d.is_dir()]

                for round_dir in round_dirs:
                    round_name = round_dir.name

                    # skip test rounds
                    if is_test_round(round_name):
                        continue

                    latency = extract_latency(round_name)
                    assert latency is not None, "found non-conforming latency round"

                    # Check what files exist
                    files = {f.name: f for f in round_dir.iterdir() if f.is_file()}

                    data_inventory.append(
                        {
                            "participant_id": participant_id,
                            "session_timestamp": session_timestamp,
                            "game": game_name,
                            "round_timestamp": get_round_name_without_latency(round_name),
                            "latency_ms": latency,
                            "has_qoe": "qoe.txt" in files,
                            "has_kb": "kb.csv" in files,
                            "has_mouse": "mouse.csv" in files,
                            "has_score_png": "score.png" in files if game_name in ('dave_the_diver', 'feeding_frenzy') else None,
                            "has_dump": "dump.txt" in files if game_name == 'half_life_2' else None,
                            "has_score_txt": "score.txt" in files if game_name in ('dave_the_diver', 'half_life_2', 'feeding_frenzy') else None,
                            "has_results": "results.csv" in files if game_name == 'fitts_law' else None,
                            "has_replay": any(".replay" in f for f in files) if game_name == 'rocket_league' else None,
                            "has_replay_json": any(".replay.json" in f for f in files) if game_name == 'rocket_league' else None,
                            "round_path": str(round_dir),
                        }
                    )

    return pd.DataFrame(data_inventory)


# Scan all data
inventory_df = scan_experiment_data()
print(f"Total rounds found: {len(inventory_df)}")
print(f"Participants: {len(inventory_df['participant_id'].unique())}")
print(f"Games: {sorted(inventory_df['game'].unique())}")
print(f"NOTE: In has_* columns, True/False indices whether data files are there where they are supposed to be, None indicates results not collected for that game type")
inventory_df.head(10)

Total rounds found: 488
Participants: 27
Games: ['dave_the_diver', 'feeding_frenzy', 'fitts_law', 'half_life_2', 'rocket_league']
NOTE: In has_* columns, True/False indices whether data files are there where they are supposed to be, None indicates results not collected for that game type


,participant_id,session_timestamp,game,round_timestamp,latency_ms,has_qoe,has_kb,has_mouse,has_score_png,has_dump,has_score_txt,has_results,has_replay,has_replay_json,round_path
0,1,2026-01-23_15-13-43,dave_the_diver,2026-01-23_15-19-01,150,True,True,True,True,None,True,None,None,None,../experiment/1/2026-01-23_15-13-43/dave_the_diver/2026-01-23_15-19-01_150ms
1,1,2026-01-23_15-13-43,dave_the_diver,2026-01-23_15-15-54,0,True,True,True,True,None,True,None,None,None,../experiment/1/2026-01-23_15-13-43/dave_the_diver/2026-01-23_15-15-54_0ms
2,1,2026-01-23_15-13-43,dave_the_diver,2026-01-23_15-17-30,75,True,True,True,True,None,True,None,None,None,../experiment/1/2026-01-23_15-13-43/dave_the_diver/2026-01-23_15-17-30_75ms
3,1,2026-01-23_15-13-43,dave_the_diver,2026-01-23_15-20-32,225,True,True,True,True,None,True,None,None,None,../experiment/1/2026-01-23_15-13-43/dave_the_diver/2026-01-23_15-20-32_225ms
4,1,2026-01-23_15-13-43,half_life_2,2026-01-23_15-23-51,75,True,True,True,None,True,True,None,None,None,../experiment/1/2026-01-23_15-13-43/half_life_2/2026-01-23_15-23-51_75ms
5,1,2026-01-23_15-13-43,half_life_2,2026-01-23_15-23-08,0,True,True,True,None,True,True,None,None,None,../experiment/1/2026-01-23_15-13-43/half_life_2/2026-01-23_15-23-08_0ms
6,1,2026-01-23_15-13-43,half_life_2,2026-01-23_15-24-33,150,True,True,True,None,True,True,None,None,None,../experiment/1/2026-01-23_15-13-43/half_life_2/2026-01-23_15-24-33_150ms
7,1,2026-01-23_15-13-43,half_life_2,2026-01-23_15-25-12,225,True,True,True,None,True,True,None,None,None,../experiment/1/2026-01-23_15-13-43/half_life_2/2026-01-23_15-25-12_225ms
8,1,2026-01-23_15-13-43,fitts_law,2026-01-23_14-56-41,0,True,True,True,None,None,None,True,None,None,../experiment/1/2026-01-23_15-13-43/fitts_law/2026-01-23_14-56-41_0ms
9,1,2026-01-23_15-13-43,fitts_law,2026-01-23_14-58-24,150,True,True,True,None,None,None,True,None,None,../experiment/1/2026-01-23_15-13-43/fitts_law/2026-01-23_14-58-24_150ms


## 2. Data Quality Report


In [4]:
def add_quality_flags(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # always-required
    df["missing_qoe"] = df["has_qoe"] == False
    df["missing_kb"] = df["has_kb"] == False
    df["missing_mouse"] = df["has_mouse"] == False

    # Feeding Frenzy / Dave the Diver
    df["ff_missing_score_png"] = (
        df["game"].isin(["feeding_frenzy", "dave_the_diver"])
        & (df["has_score_png"] == False)
    )
    df["ff_missing_score_txt"] = (
        df["game"].isin(["feeding_frenzy", "dave_the_diver"])
        & (df["has_score_txt"] == False)
    )

    # Half-Life 2
    df["hl2_missing_dump"] = (
        (df["game"] == "half_life_2") & (df["has_dump"] == False)
    )
    df["hl2_missing_score_txt"] = (
        (df["game"] == "half_life_2") & (df["has_score_txt"] == False)
    )

    # Fitts' Law
    df["fitts_missing_results"] = (
        (df["game"] == "fitts_law") & (df["has_results"] == False)
    )

    # Rocket League
    df["rl_missing_replay"] = (
        (df["game"] == "rocket_league") & (df["has_replay"] == False)
    )
    df["rl_missing_replay_json"] = (
        (df["game"] == "rocket_league") & (df["has_replay_json"] == False)
    )

    # Parsed-without-raw (pipeline bugs)
    df["score_txt_without_png"] = (
        df["game"].isin(["feeding_frenzy", "dave_the_diver"])
        & (df["has_score_txt"] == True)
        & (df["has_score_png"] == False)
    )

    df["replay_json_without_replay"] = (
        (df["game"] == "rocket_league")
        & (df["has_replay_json"] == True)
        & (df["has_replay"] == False)
    )

    df["hl2_score_without_dump"] = (
        (df["game"] == "half_life_2")
        & (df["has_score_txt"] == True)
        & (df["has_dump"] == False)
    )

    return df

inventory_df = add_quality_flags(inventory_df)

In [5]:
quality_cols = [
    c for c in inventory_df.columns
    if "missing_" in c or "without_" in c
]

print("high level audit: count by failure type")
inventory_df[quality_cols].sum().sort_values(ascending=False)

high level audit: count by failure type


rl_missing_replay             4
rl_missing_replay_json        4
missing_qoe                   1
missing_kb                    0
missing_mouse                 0
ff_missing_score_txt          0
ff_missing_score_png          0
hl2_missing_score_txt         0
hl2_missing_dump              0
fitts_missing_results         0
score_txt_without_png         0
replay_json_without_replay    0
hl2_score_without_dump        0
dtype: int64

In [6]:
print("rounds missing qoe")
display(
    inventory_df.query(
        "missing_qoe"
    )
)
print("rocket league rounds missing replay or parsed replay")
display(
    inventory_df.query(
        "game == 'rocket_league' and (rl_missing_replay or rl_missing_replay_json)"
    )
)

rounds missing qoe


,participant_id,session_timestamp,game,round_timestamp,latency_ms,has_qoe,has_kb,has_mouse,has_score_png,has_dump,has_score_txt,has_results,has_replay,has_replay_json,round_path,missing_qoe,missing_kb,missing_mouse,ff_missing_score_png,ff_missing_score_txt,hl2_missing_dump,hl2_missing_score_txt,fitts_missing_results,rl_missing_replay,rl_missing_replay_json,score_txt_without_png,replay_json_without_replay,hl2_score_without_dump
386,4,2026-01-24_12-23-49,dave_the_diver,2026-01-24_12-19-54,150,False,True,True,True,None,True,None,None,None,../experiment/4/2026-01-24_12-23-49/dave_the_diver/2026-01-24_12-19-54_150ms,True,False,False,False,False,False,False,False,False,False,False,False,False


rocket league rounds missing replay or parsed replay


,participant_id,session_timestamp,game,round_timestamp,latency_ms,has_qoe,has_kb,has_mouse,has_score_png,has_dump,has_score_txt,has_results,has_replay,has_replay_json,round_path,missing_qoe,missing_kb,missing_mouse,ff_missing_score_png,ff_missing_score_txt,hl2_missing_dump,hl2_missing_score_txt,fitts_missing_results,rl_missing_replay,rl_missing_replay_json,score_txt_without_png,replay_json_without_replay,hl2_score_without_dump
131,16,2026-01-30_14-58-25,rocket_league,2026-01-30_15-26-46,150,True,True,True,None,None,None,None,False,False,../experiment/16/2026-01-30_14-58-25/rocket_league/2026-01-30_15-26-46_150ms,False,False,False,False,False,False,False,False,True,True,False,False,False
293,24,2026-02-05_17-40-10,rocket_league,2026-02-05_17-57-28,150,True,True,True,None,None,None,None,False,False,../experiment/24/2026-02-05_17-40-10/rocket_league/2026-02-05_17-57-28_150ms,False,False,False,False,False,False,False,False,True,True,False,False,False
312,25,2026-02-06_14-59-43,rocket_league,2026-02-06_15-14-30,225,True,True,True,None,None,None,None,False,False,../experiment/25/2026-02-06_14-59-43/rocket_league/2026-02-06_15-14-30_225ms,False,False,False,False,False,False,False,False,True,True,False,False,False
472,9,2026-01-28_14-02-39,rocket_league,2026-01-28_14-10-59,225,True,True,True,None,None,None,None,False,False,../experiment/9/2026-01-28_14-02-39/rocket_league/2026-01-28_14-10-59_225ms,False,False,False,False,False,False,False,False,True,True,False,False,False


In [7]:
# Filter to measured rounds only (exclude test rounds)
measured_df = inventory_df.copy()

print("=" * 80)
print("DATA QUALITY REPORT")
print("=" * 80)

# Expected data structure
participants = measured_df["participant_id"].unique()
games = list(GAME_NAMES.keys())
latencies = LATENCIES
expected_rounds = len(participants) * len(games) * len(latencies)

print(
    f"\nExpected: {len(participants)} participants × {len(games)} games × {len(latencies)} latencies = {expected_rounds} rounds"
)
print(f"Found: {len(measured_df)} measured rounds")
print(f"Missing: {expected_rounds - len(measured_df)} rounds")

# Identify missing data
print("\n" + "=" * 80)
print("MISSING DATA DETAILS")
print("=" * 80)

for participant_id in sorted(participants):
    participant_data = measured_df[measured_df["participant_id"] == participant_id]
    for game in GAME_NAMES.keys():
        game_data = participant_data[participant_data["game"] == game]
        if len(game_data) < 4:
            missing_latencies = set(latencies) - set(game_data["latency_ms"])
            print(f"Participant {participant_id}, {game}: {len(game_data)}/{len(latencies)} rounds (missing: {missing_latencies})")

# Check completeness by participant and game
print("\n" + "=" * 80)
print("COMPLETENESS BY PARTICIPANT AND GAME")
print("=" * 80)

completeness = measured_df.groupby(["participant_id", "game"]).size().unstack(fill_value=0)
print(completeness)

DATA QUALITY REPORT

Expected: 27 participants × 5 games × 4 latencies = 540 rounds
Found: 488 measured rounds
Missing: 52 rounds

MISSING DATA DETAILS
Participant 1, rocket_league: 0/4 rounds (missing: {0, 225, 75, 150})
Participant 2, dave_the_diver: 0/4 rounds (missing: {0, 225, 75, 150})
Participant 5, fitts_law: 0/4 rounds (missing: {0, 225, 75, 150})
Participant 5, rocket_league: 0/4 rounds (missing: {0, 225, 75, 150})
Participant 5, half_life_2: 0/4 rounds (missing: {0, 225, 75, 150})
Participant 7, dave_the_diver: 0/4 rounds (missing: {0, 225, 75, 150})
Participant 10, rocket_league: 0/4 rounds (missing: {0, 225, 75, 150})
Participant 10, dave_the_diver: 0/4 rounds (missing: {0, 225, 75, 150})
Participant 13, rocket_league: 0/4 rounds (missing: {0, 225, 75, 150})
Participant 18, rocket_league: 0/4 rounds (missing: {0, 225, 75, 150})
Participant 21, dave_the_diver: 0/4 rounds (missing: {0, 225, 75, 150})
Participant 23, dave_the_diver: 0/4 rounds (missing: {0, 225, 75, 150})
Par

## 3. Extract Performance Metrics

Extract game-specific performance metrics from each round.


In [8]:
def parse_fitts_law_data(round_path: pathlib.Path) -> dict:
    """Parse Fitts Law results.csv file."""
    results_file = Path(round_path) / "results.csv"
    if not results_file.exists():
        raise ValueError(f"fitts law results.csv not found at {results_file}")

    df = pd.read_csv(results_file)
    # filter to successful, non-ignored attempts
    valid = df[(df["is_success"] == True) & (df["ignored"] == False)]
    if len(valid) == 0:
        raise ValueError(f"invalid results at {results_file}")

    # Calculate throughput (Index of Difficulty / Movement Time)
    # Throughput is a standard Fitts Law metric
    valid_with_id = valid[valid["ID_bits"] > 0]
    if len(valid_with_id) == 0:
        raise ValueError(f"no valid attempts with ID > 0 at {results_file}")

    return {
        "mean_time_ms": valid_with_id["time_ms"].mean(),
        "median_time_ms": valid_with_id["time_ms"].median(),
        "throughput": (valid_with_id["ID_bits"] / (valid_with_id["time_ms"] / 1000)).mean(),
        "n_trials": len(valid_with_id),
    }

def is_number_with_comma(s):
    # Remove commas from the string
    s_without_comma = s.replace(',', '')

    # Try converting the result to a float
    try:
        float(s_without_comma)
        # Check if the original string had valid comma placement (optional, depends on specific rules)
        # For simplicity, this function just checks if it's a number once commas are gone.
        return True
    except ValueError:
        return False


def parse_score_file(round_path: pathlib.Path) -> float:
    """Parse generic score.txt file."""
    score_file = Path(round_path) / "score.txt"
    if not score_file.exists():
        raise ValueError(f"score.txt not found at {score_file}")

    content = score_file.read_text().strip()
    # Try to extract numeric value
    # Handle formats like "700", "13", "2.7/5.0kg"

    # First try: simple integer - feeding frenzy, half-life 2
    if is_number_with_comma(content):
        return float(content.replace(',', ''))

    # Second try: extract first number (for formats like "2.7/5.0kg") - dave the diver
    match = re.search(r"([0-9]+\.?[0-9]*)", content)
    if not match:
        raise ValueError(f"wrong score format in {score_file}: '{content}'")

    return float(match.group(0))


def parse_rocket_league_replay(round_path: pathlib.Path) -> dict:
    """Parse Rocket League replay JSON file."""
    round_dir = Path(round_path)
    replay_files = list(round_dir.glob("*.replay.json"))

    if not replay_files:
        # SPECIAL CASES: 4 cases that are missing rocket league mentioned above
        if "../experiment/16/2026-01-30_14-58-25/rocket_league/2026-01-30_15-26-46_150ms" in str(round_path):
            return {
                "score": 0,
                "goals": 0,
                "saves": 0,
                "assists": 0,
                "shots": 0
            }
        elif "../experiment/24/2026-02-05_17-40-10/rocket_league/2026-02-05_17-57-28_150ms" in str(round_path):
            return {
                "score": 24,
                "goals": 0,
                "saves": 0,
                "assists": 0,
                "shots": 0
            }
        elif "../experiment/25/2026-02-06_14-59-43/rocket_league/2026-02-06_15-14-30_225ms" in str(round_path):
            return {
                "score": 8,
                "goals": 0,
                "saves": 0,
                "assists": 0,
                "shots": 0
            }
        elif "./experiment/9/2026-01-28_14-02-39/rocket_league/2026-01-28_14-10-59_225ms" in str(round_path):
            return {
                "score": 46,
                "goals": 0,
                "saves": 0,
                "assists": 0,
                "shots": 2
            }
        raise ValueError(f"no replay JSON file found in {round_dir}")

    with replay_files[0].open("r") as f:
        replay_data = json.load(f)

    # Extract relevant stats
    result = {}

    props = replay_data["properties"]
    # Try to get player stats (primary method)
    player_stats = props["PlayerStats"][0]  # Get first player (should be the participant)
    result["score"] = player_stats["Score"]
    result["goals"] = player_stats["Goals"]
    result["saves"] = player_stats["Saves"]
    result["assists"] = player_stats["Assists"]
    result["shots"] = player_stats["Shots"]

    # Fallback: Try to get team score
    # elif "Team1Score" in props:
    #     result["goals"] = props["Team1Score"]
    #     result["score"] = props["Team1Score"]

    return result


def extract_performance_metrics(measured_df: pd.DataFrame) -> pd.DataFrame:
    """Extract all performance metrics."""
    performance_data = []

    for _, row in measured_df.iterrows():
        participant_id = row["participant_id"]
        game = row["game"]
        latency = row["latency_ms"]
        round_path = pathlib.Path(row["round_path"])

        metrics = {"participant_id": participant_id, "game": game, "latency_ms": latency}

        # Game-specific parsing
        if game == "fitts_law":
            fitts_data = parse_fitts_law_data(round_path)
            metrics.update(fitts_data)
            metrics["score"] = fitts_data["throughput"]  # primary metric

        elif game == "rocket_league":
            replay_data = parse_rocket_league_replay(round_path)
            metrics.update(replay_data) # score is already in replay_data
            # Extract score - try goals first, then score, then use 0 as fallback
            # score_value = replay_data.get("goals")
            # if score_value is None:
            #     score_value = replay_data.get("score")
            # Ensure score is a number, not a dict or list
            # if score_value is not None and isinstance(score_value, (int, float)):
            #     metrics["score"] = score_value
        else:
            # For other games, use score.txt
            metrics["score"] = parse_score_file(round_path)

        if "score" not in metrics:
            raise ValueError(f"no score found for participant {participant_id}, game {game}, latency {latency}ms at {round_path}")

        performance_data.append(metrics)

    return pd.DataFrame(performance_data)


# Extract performance metrics
performance_df = extract_performance_metrics(measured_df)
print(f"\nExtracted performance data for {len(performance_df)} rounds")
performance_df.head(20)


Extracted performance data for 488 rounds


,participant_id,game,latency_ms,score,mean_time_ms,median_time_ms,throughput,n_trials,goals,saves,assists,shots
0,1,dave_the_diver,150,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dave_the_diver,0,2.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,dave_the_diver,75,2.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,dave_the_diver,225,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,half_life_2,75,15.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,half_life_2,0,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,half_life_2,150,11.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,half_life_2,225,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,fitts_law,0,4.790658,809.677778,769.50,4.790658,27.0,NaN,NaN,NaN,NaN
9,1,fitts_law,150,2.972112,1673.075000,1585.15,2.972112,12.0,NaN,NaN,NaN,NaN


## 4. Extract QoE Data


In [9]:
def parse_qoe_file(round_path: pathlib.Path) -> dict:
    """Parse QoE questionnaire data."""
    qoe_file = Path(round_path) / "qoe.txt"
    if not qoe_file.exists():
        raise ValueError(f"qoe.txt not found at {qoe_file}")

    content = qoe_file.read_text().strip()
    # Format: "QoE: 5.0 : 1,"
    # quality_rating : acceptability (1 for Yes or 2 for No)
    match = re.search(r"QoE:\s*([0-9.]+)\s*:\s*([0-9]+)", content)
    if not match:
        raise ValueError(f"wrong QoE format in {qoe_file}: '{content}'")

    quality = float(match.group(1))
    if not (0 <= quality <= 5):
        raise ValueError(f"quality rating out of range in {qoe_file}: '{quality}'")

    acceptable = int(match.group(2))
    if acceptable not in (1, 2):
        raise ValueError(f"acceptability rating must be 1 or 2 in {qoe_file}: '{acceptable}'")

    return {"quality_rating": quality, "acceptable": 1 if acceptable == 1 else 0}


def extract_qoe_data(measured_df: pd.DataFrame) -> pd.DataFrame:
    """Extract all QoE data."""
    qoe_data = []

    for _, row in measured_df.iterrows():
        # skip rounds that don't have QoE data (1 round)
        if not row["has_qoe"]:
            continue

        qoe = parse_qoe_file(row["round_path"])
        qoe_data.append(
            {
                "participant_id": row["participant_id"],
                "game": row["game"],
                "latency_ms": row["latency_ms"],
                "quality_rating": qoe["quality_rating"],
                "acceptable": qoe["acceptable"],
            }
        )

    return pd.DataFrame(qoe_data)


# Extract QoE data
qoe_df = extract_qoe_data(measured_df)
print(f"\nExtracted QoE data for {len(qoe_df)} rounds")
print(f"\nQoE Summary:")
print(qoe_df.groupby("latency_ms")[["quality_rating", "acceptable"]].agg(["mean", "std", "count"]))
qoe_df.head(20)


Extracted QoE data for 487 rounds

QoE Summary:
           quality_rating                 acceptable                
                     mean       std count       mean       std count
latency_ms                                                          
0                4.485000  0.774818   122   0.967213  0.178813   122
75               3.792623  0.961848   122   0.836066  0.371743   122
150              3.026446  1.297097   121   0.595041  0.492925   121
225              2.321311  1.154741   122   0.401639  0.492251   122


,participant_id,game,latency_ms,quality_rating,acceptable
0,1,dave_the_diver,150,4.0,1
1,1,dave_the_diver,0,5.0,1
2,1,dave_the_diver,75,5.0,1
3,1,dave_the_diver,225,3.2,0
4,1,half_life_2,75,4.5,1
5,1,half_life_2,0,4.5,1
6,1,half_life_2,150,2.2,0
7,1,half_life_2,225,1.5,0
8,1,fitts_law,0,5.0,1
9,1,fitts_law,150,1.1,0


## 5. Data Cleaning and Merging


In [10]:
# Merge performance and QoE data
combined_df = performance_df.merge(qoe_df, on=["participant_id", "game", "latency_ms"], how="left")

print(f"Combined dataset: {len(combined_df)} rows")
print(f"\nData by game:")
print(combined_df.groupby("game").size())

# Add game display names
combined_df["game_display"] = combined_df["game"].map(GAME_NAMES)

# Check for missing QoE data
missing_qoe = combined_df["quality_rating"].isna().sum()
if missing_qoe > 0:
    print(f"\nWarning: {missing_qoe} rounds missing QoE data")

combined_df.head(20)

Combined dataset: 488 rows

Data by game:
game
dave_the_diver     84
feeding_frenzy    108
fitts_law         104
half_life_2       104
rocket_league      88
dtype: int64



,participant_id,game,latency_ms,score,mean_time_ms,median_time_ms,throughput,n_trials,goals,saves,assists,shots,quality_rating,acceptable,game_display
0,1,dave_the_diver,150,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,Dave the Diver
1,1,dave_the_diver,0,2.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,Dave the Diver
2,1,dave_the_diver,75,2.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,Dave the Diver
3,1,dave_the_diver,225,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2,0.0,Dave the Diver
4,1,half_life_2,75,15.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,1.0,Half-Life 2
5,1,half_life_2,0,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,1.0,Half-Life 2
6,1,half_life_2,150,11.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2,0.0,Half-Life 2
7,1,half_life_2,225,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,0.0,Half-Life 2
8,1,fitts_law,0,4.790658,809.677778,769.50,4.790658,27.0,NaN,NaN,NaN,NaN,5.0,1.0,Fitts Law
9,1,fitts_law,150,2.972112,1673.075000,1585.15,2.972112,12.0,NaN,NaN,NaN,NaN,1.1,0.0,Fitts Law


## 6. Create Normalized Scores

For cross-game comparisons, we'll create z-scored performance metrics.


In [11]:
# Calculate z-scores within each game
# Handle cases where std = 0 (all scores same) or std = NaN
def safe_zscore(x: pd.Series) -> pd.Series:
    std = x.std()
    if std == 0 or pd.isna(std):
        return pd.Series(0, index=x.index)  # All values same, z-score = 0
    return (x - x.mean()) / std

combined_df["score_z"] = combined_df.groupby("game")["score"].transform(safe_zscore)

# Also calculate per-participant normalized scores (relative to their baseline)
def safe_pct_of_max(x: pd.Series) -> pd.Series:
    max_val = x.max()
    if max_val > 0:
        return (x / max_val) * 100
    return pd.Series(np.nan, index=x.index)

combined_df["score_pct_of_baseline"] = combined_df.groupby(["participant_id", "game"])["score"].transform(safe_pct_of_max)

print("\nScore statistics by latency:")
print(combined_df.groupby("latency_ms")["score_z"].agg(["mean", "std", "count"]))


Score statistics by latency:
                mean       std  count
latency_ms                           
0           0.749896  0.970266    122
75          0.279413  0.800100    122
150        -0.309324  0.785916    122
225        -0.719984  0.726399    122


## 7. Export Processed Data


In [12]:
# Export to CSV
output_file = OUTPUT_DIR / "combined_data.csv"
combined_df.to_csv(output_file, index=False)
print(f"Saved combined data to: {output_file}")

# Export separate files for convenience
performance_df.to_csv(OUTPUT_DIR / "performance_data.csv", index=False)
qoe_df.to_csv(OUTPUT_DIR / "qoe_data.csv", index=False)
inventory_df.to_csv(OUTPUT_DIR / "data_inventory.csv", index=False)

print(f"\nExported files:")
print(f"  - combined_data.csv ({len(combined_df)} rows)")
print(f"  - performance_data.csv ({len(performance_df)} rows)")
print(f"  - qoe_data.csv ({len(qoe_df)} rows)")
print(f"  - data_inventory.csv ({len(inventory_df)} rows)")

Saved combined data to: ../analysis/processed_data/combined_data.csv

Exported files:
  - combined_data.csv (488 rows)
  - performance_data.csv (488 rows)
  - qoe_data.csv (487 rows)
  - data_inventory.csv (488 rows)


## 8. Data Summary Statistics


In [13]:
print("=" * 80)
print("FINAL DATA SUMMARY")
print("=" * 80)

print(f"\nTotal valid observations: {len(combined_df)}")
print(f"Participants: {len(combined_df['participant_id'].unique())}")
print(f"Games: {len(combined_df['game'].unique())}")
print(f"Latency conditions: {sorted(combined_df['latency_ms'].unique())}")

print("\n" + "=" * 80)
print("OBSERVATIONS PER GAME")
print("=" * 80)
game_summary = combined_df.groupby("game_display").agg(
    {"participant_id": "nunique", "score": ["count", "mean", "std"], "quality_rating": ["mean", "std"]}
)
print(game_summary)

print("\n" + "=" * 80)
print("METRICS BY LATENCY CONDITION")
print("=" * 80)
latency_summary = combined_df.groupby("latency_ms").agg(
    {"score_z": ["mean", "std"], "quality_rating": ["mean", "std"], "acceptable": "mean"}
)
print(latency_summary)

print("\n" + "=" * 80)
print("GAME × LATENCY BREAKDOWN")
print("=" * 80)
game_latency = combined_df.pivot_table(index="game_display", columns="latency_ms", values="score", aggfunc="count")
print(game_latency)

FINAL DATA SUMMARY

Total valid observations: 488
Participants: 27
Games: 5
Latency conditions: [np.int64(0), np.int64(75), np.int64(150), np.int64(225)]

OBSERVATIONS PER GAME
               participant_id score                         quality_rating  \
                      nunique count        mean         std           mean   
game_display                                                                 
Dave the Diver             21    84    2.226190    1.172077       3.906024   
Feeding Frenzy             27   108  493.333333  359.100642       3.587685   
Fitts Law                  26   104    3.408102    1.181863       2.799038   
Half-Life 2                26   104   12.240385    3.337039       3.145192   
Rocket League              22    88  169.806818  128.435489       3.743182   

                          
                     std  
game_display              
Dave the Diver  1.095484  
Feeding Frenzy  1.149760  
Fitts Law       1.446421  
Half-Life 2     1.443491  
Rocket Le